<a href="https://colab.research.google.com/github/casjunior93/projeto-integrador-iv/blob/main/PI_IV_Rede_neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas fundamentais

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import scipy.stats as stats

#Configura o pandas para não exibir resultados em notação científica
pd.options.display.float_format = '{:.2f}'.format

#Semente aleatória do Numpy
np.random.seed(42)

#Data local
import locale
# Colab não suporta
#locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

# URL base da api de dados

In [2]:
url_base = 'https://univesp-analytics.vercel.app/api/get'

# Lendo Json com Pandas

In [3]:
import urllib.request, json 
from pandas.io.json import json_normalize

pagina = 1
limite = 10000

In [4]:
%%time
fim_iteracao_api = False
df_dados_api_original = pd.DataFrame([])

print(f'Início das requisições')
while fim_iteracao_api == False:
  with urllib.request.urlopen(url_base + '?page=' + str(pagina) + '&limite=' + str(limite)) as url:
    data = json.loads(url.read().decode())
    df = pd.DataFrame(json_normalize(data))
    if len(data) == 0:
      print(f'Fim. Total de {pagina - 1} páginas.')
      fim_iteracao_api = True
    else:
      df_dados_api_original = pd.concat([df_dados_api_original, df], ignore_index = True)
      print('.',end='')
      pagina += 1

Início das requisições
........Fim. Total de 8 páginas.
CPU times: user 1.62 s, sys: 139 ms, total: 1.76 s
Wall time: 19.7 s


Cópia dos dados da api

In [5]:
df_dados_api = df_dados_api_original.copy()

Dimensão dos dados - Linhas, colunas

In [6]:
df_dados_api.shape

(77370, 9)

# Visualizando dados originais

Mudando ordenação dos dados

In [7]:
df_dados_api.sort_index(ascending=False, inplace=True, ignore_index=True)

In [8]:
df_dados_api.head()

,_id,temp_ar,umid_ar,type,local,time.time_js,time.data,time.hora,teste
0,63fea0720dd58484fc266c80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.00
1,63ffdbb91a4eb0dfdd6a11a7,2,3,teste,NaN,2023-03-01T23:11:53.036Z,01/03/2023,20:11:53,NaN
2,63ffdbde1a4eb0dfdd6a11a8,10,10,teste,NaN,2023-03-01T23:12:30.063Z,01/03/2023,20:12:30,NaN
3,63ffdea6722bef68acc5bb72,333,333,teste,NaN,2023-03-01T23:24:21.438Z,01/03/2023,23:24:21,NaN
4,63ffed2b38c8784e07b788f2,15,15,teste,NaN,01/03/2023 21:26:19,01/03/2023,21:26:19,NaN


In [9]:
df_dados_api.tail()

,_id,temp_ar,umid_ar,type,local,time.time_js,time.data,time.hora,teste
77365,64486c27fc14cad3669fbf8c,24.60,66.70,teste,semAr,"25/04/2023, 21:11:19","25/04/2023,",21:11:19,NaN
77366,64486d34fc14cad3669fbf8d,24.70,55.90,teste,comAr,"25/04/2023, 21:15:48","25/04/2023,",21:15:48,NaN
77367,64486d55fc14cad3669fbf8e,24.60,66.50,teste,semAr,"25/04/2023, 21:16:21","25/04/2023,",21:16:21,NaN
77368,64486e62fc14cad3669fbf8f,24.70,56.40,teste,comAr,"25/04/2023, 21:20:50","25/04/2023,",21:20:50,NaN
77369,64486e83fc14cad3669fbf90,24.60,66.40,teste,semAr,"25/04/2023, 21:21:23","25/04/2023,",21:21:23,NaN


Excluindo colunas desnecessárias

In [10]:
del df_dados_api['teste']
del df_dados_api['_id']
del df_dados_api['type']
del df_dados_api['time.time_js']
df_dados_api.head()

,temp_ar,umid_ar,local,time.data,time.hora
0,NaN,NaN,NaN,NaN,NaN
1,2,3,NaN,01/03/2023,20:11:53
2,10,10,NaN,01/03/2023,20:12:30
3,333,333,NaN,01/03/2023,23:24:21
4,15,15,NaN,01/03/2023,21:26:19


Coluna local

In [16]:
df_dados_api.local.value_counts()

semAr    19527
comAr     2427
API          2
teste        1
123          1
Name: local, dtype: int64

In [17]:
values = {"local": "comAr"}
df_dados_api.fillna(value=values, inplace=True)

In [18]:
df_dados_api.local.value_counts()

comAr    57839
semAr    19527
API          2
teste        1
123          1
Name: local, dtype: int64